In [1]:
#conda activate comp-methods3

In [12]:
#!pip install anndata
#!pip install hdf5plugin
#!pip install anndata2ri
#!pip install rpy2
#!pip install episcanpy

In [45]:
import anndata as ad
import hdf5plugin

# preprocessed data from the episcanpy tutorial
adata = ad.read("Adata.h5ad")

In [3]:
import anndata2ri
import logging
from scipy.sparse import issparse

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%reload_ext rpy2.ipython

In [33]:
import scipy

data_mat = adata.X.T

# convert to CSC if possible. See https://github.com/MarioniLab/scran/issues/70
if scipy.sparse.issparse(data_mat):
    if data_mat.nnz > 2**31 - 1:
        data_mat = data_mat.tocoo()
    else:
        data_mat = data_mat.tocsc()
ro.globalenv["data_mat"] = data_mat
ro.globalenv["input_groups"] = adata_pp.obs["leiden"]

In [34]:
%%R -i data_mat -o size_factors
library(scran)
library(BiocParallel)

size_factors = sizeFactors(
    computeSumFactors(
        SingleCellExperiment(
            list(counts=data_mat)), 
            clusters = input_groups,
            min.mean = 0.1,
            BPPARAM = MulticoreParam()
    )
)

In [40]:
from scipy.sparse import csr_matrix, issparse

adata.obs["size_factors"] = size_factors

In [ ]:
scran = adata.X / adata.obs["size_factors"].values[:, None]
adata.layers["scran_normalization"] = csr_matrix(epi.pp.log1p(scran))

In [41]:
adata.write_h5ad(
    "Normalized.h5ad",
    compression=hdf5plugin.FILTERS["zstd"]
)

In [ ]:
A = adata.layers["scran_normalization"]

pd.DataFrame(A.toarray())